In [1]:
import pandas as pd

df = pd.read_parquet('train.parquet')

In [2]:
import spacy

from spacy.training.iob_utils import biluo_to_iob, doc_to_biluo_tags
from tqdm.autonotebook import tqdm
tqdm.pandas()

df.trigger_words = df.trigger_words.apply(lambda x: [] if x is None else x)
df['target'] = df.trigger_words.apply(lambda x: [[y[0], y[1], 'TRIGGER'] for y in x])

def resolve_overlapping_spans(spans):
    if not spans:
        return []
    spans = sorted(spans, key=lambda x: x[0])  # Sort by start index
    resolved = [spans[0]]
    for current in spans[1:]:
        last = resolved[-1]
        if current[0] < last[1]:  # Overlap
            new_span = (last[0], max(last[1], current[1]), 'TRIGGER')
            resolved[-1] = new_span
            print('resolved')
        else:
            resolved.append(current)
    return resolved

df['target'] = df.target.apply(resolve_overlapping_spans)

nlp = spacy.blank("xx")

def convert_to_conll(row):
    data = {
        "text": row['content'],
        "label": row['target']
    }
    doc = nlp(data["text"])
    ents = []
    for start, end, label in data["label"]:
        span = doc.char_span(start, end, label=label)
        if span is not None:
            ents.append(span)
        else:
            pass
        #TODO fix not align to token case
        '''
            print(
                "Skipping span (does not align to tokens):",
                start,
                end,
                label,
                doc.text[start:end],
            )
        '''
    doc.ents = ents
    return {
        'tokens': list([t.text for t in doc]),
        'labels': list(biluo_to_iob(doc_to_biluo_tags(doc)))
    }

df['conll'] = df.progress_apply(convert_to_conll, axis=1)

resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved
resolved


/tmp/ipykernel_65684/1739179680.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


  0%|          | 0/3822 [00:00<?, ?it/s]

In [3]:
import numpy as np


df['is_valid'] = np.random.binomial(1, 0.2, df.shape[0])

In [4]:
label2id = {'O': 0, 'B-TRIGGER': 1, 'I-TRIGGER': 2}

df['tokens'] = df.conll.str['tokens']
df['ner_tags'] = df.conll.str['labels'].apply(lambda x: [label2id[t] for t in x])

df_train = df[df.is_valid == 0]
df_valid = df[df.is_valid == 1]

In [5]:
df_train[['tokens', 'ner_tags']].to_json(
    './data/train_processed.json', orient='records', lines=True)
df_valid[['tokens', 'ner_tags']].to_json(
    './data/valid_processed.json', orient='records', lines=True)

In [6]:
from datasets import load_dataset

raw_datasets_ua = load_dataset(
    "json",
    data_files={
        'train': './data/train_processed.json',
        'val': './data/valid_processed.json'
    }
)
raw_datasets_ua

Generating train split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 3021
    })
    val: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 801
    })
})

In [7]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

PRETRAINED_MODEL = 'google/gemma-2-9b-it'
MAX_LENGTH = 1024

id2label = {v: k for k, v in label2id.items()}

#model = AutoModelForTokenClassification.from_pretrained(
#    'bert-base-multilingual-cased',
#    id2label=id2label,
#    label2id=label2id,
#)
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL)

In [8]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)


def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [9]:
tokenized_datasets_ua = raw_datasets_ua.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets_ua["train"].column_names,
)

Map:   0%|          | 0/3021 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/801 [00:00<?, ? examples/s]

In [10]:
from transformers import AdamW, get_linear_schedule_with_warmup


EPOCHS = 3

In [11]:
import torch
from transformers import Gemma2ForTokenClassification, BitsAndBytesConfig
from peft import get_peft_config, prepare_model_for_kbit_training, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType



model = Gemma2ForTokenClassification.from_pretrained(
    PRETRAINED_MODEL,
    id2label=id2label,
    label2id=label2id,
    torch_dtype=torch.bfloat16
)

lora_config = LoraConfig(
    r=32,  # the dimension of the low-rank matrices
    lora_alpha=16, # scaling factor for LoRA activations vs pre-trained weight activations
    lora_dropout=0.05, 
    bias='none',
    inference_mode=False,
    task_type=TaskType.TOKEN_CLS,
    target_modules=['o_proj', 'v_proj', "q_proj", "k_proj", "gate_proj"]
) 

model = get_peft_model(model, lora_config)
# Trainable Parameters
model.print_trainable_parameters()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of Gemma2ForTokenClassification were not initialized from the model checkpoint at google/gemma-2-9b-it and are newly initialized: ['score.bias', 'score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 59,877,891 || all params: 9,301,594,630 || trainable%: 0.6437


In [12]:
import os
import random
import numpy as np
import torch

def set_seeds(seed):
    """Set seeds for reproducibility """
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        

set_seeds(seed=42)

In [13]:
import os
from transformers import (AutoTokenizer, TrainingArguments, Trainer,
                          AutoModelForSequenceClassification, DataCollatorWithPadding)
from sklearn.metrics import f1_score
import numpy as np


import evaluate
import numpy as np

metric = evaluate.load("seqeval")

label_names = list(label2id.keys())

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }


train_args = TrainingArguments(
    output_dir='model_checkpoints_gemma2_qlora',
    logging_dir='./model_logs_gemma2_qlora',
    learning_rate=2e-5,
    weight_decay=0.01,
    lr_scheduler_type='cosine',
    warmup_ratio=0.0,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    bf16=True,
    report_to="wandb",
    optim='adamw_torch',
    eval_strategy='steps',
    save_strategy="steps",
    eval_steps=200,
    logging_steps=20,
    save_steps=200,
    save_total_limit=2,
    metric_for_best_model='f1',
    greater_is_better=True,
    load_best_model_at_end=True,
)

In [14]:
import wandb

# Initialize with team/entity
wandb.init(
    project="unlp-span-ident-task",
    entity="bazdyrev99-igor-sikorsky-kyiv-polytechnic-institute", 
    name='gemma2-9b-baseline'
)

wandb: Currently logged in as: bazdyrev99 (bazdyrev99-igor-sikorsky-kyiv-polytechnic-institute). Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [16]:
trainer = Trainer(
    model=model, 
    args=train_args, 
    train_dataset=tokenized_datasets_ua["train"],
    eval_dataset=tokenized_datasets_ua["val"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

/tmp/ipykernel_65684/382105683.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
200,0.614000,0.487971,0.002915,0.007453,0.004191,0.799612
400,0.429200,0.454077,0.006406,0.019255,0.009614,0.801946
600,0.435200,0.447057,0.005008,0.009938,0.006660,0.810892
800,0.411000,0.438858,0.006471,0.014907,0.009024,0.811502
1000,0.417700,0.442918,0.006082,0.014907,0.008639,0.810118


The 'batch_size' argument of HybridCache is deprecated and will be removed in v4.49. Use the more precisely named 'max_batch_size' argument instead.
The 'batch_size' attribute of HybridCache is deprecated and will be removed in v4.49. Use the more precisely named 'self.max_batch_size' attribute instead.

KeyboardInterrupt



In [17]:
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = "./model_checkpoints_gemma2_qlora/checkpoint-1000/"
token_classifier = pipeline(
    "token-classification", model=model_checkpoint, aggregation_strategy="simple"
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of Gemma2ForTokenClassification were not initialized from the model checkpoint at google/gemma-2-9b-it and are newly initialized: ['score.bias', 'score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0


In [20]:
token_classifier.model = trainer.model

In [26]:
preds = token_classifier.predict(df_valid.content.tolist())

In [28]:
val_sub = [str([(p['start'], p['end']) for p in row]) for row in preds]

In [29]:
from copy import deepcopy

def safe_string(row):
    if row is None:
        return '[]'
    else:
        return str([(s[0], s[1]) for s in row])

valid_sub = deepcopy(df_valid)
valid_sub['trigger_words'] = valid_sub.trigger_words.apply(safe_string)
valid_sub_gt = deepcopy(valid_sub[['id', 'trigger_words']])
valid_sub_hat = deepcopy(valid_sub[['id', 'trigger_words']])
valid_sub_hat['trigger_words'] = val_sub

In [32]:
score(solution=valid_sub_gt, submission=valid_sub_hat, row_id_column_name='id')

0.45105001521761173

In [33]:
test = pd.read_csv('test.csv')

In [34]:
preds_test = token_classifier.predict(test.content.tolist())

In [35]:
test_sub = [str([(p['start'], p['end']) for p in row]) for row in preds_test]

In [36]:
ss = pd.read_csv('sample_submission.csv')

In [37]:
ss['trigger_words'] = test_sub

In [38]:
ss.to_csv('submissions/gemma2-9b-cv0.451.csv', index=False)

In [31]:
import pandas as pd
import pandas.api.types
from sklearn.metrics import f1_score
import ast


class ParticipantVisibleError(Exception):
    """Custom exception for participant-visible errors."""
    pass


def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str) -> float:
    """
    Compute span-level F1 score based on overlap.

    Parameters:
    - solution (pd.DataFrame): Ground truth DataFrame with row ID and token labels.
    - submission (pd.DataFrame): Submission DataFrame with row ID and token labels.
    - row_id_column_name (str): Column name for the row identifier.

    Returns:
    - float: The token-level weighted F1 score.

    Example:
    >>> solution = pd.DataFrame({
    ...     "id": [1, 2, 3],
    ...     "trigger_words": [[(612, 622), (725, 831)], [(300, 312)], []]
    ... })
    >>> submission = pd.DataFrame({
    ...     "id": [1, 2, 3],
    ...     "trigger_words": [[(612, 622), (700, 720)], [(300, 312)], [(100, 200)]]
    ... })
    >>> score(solution, submission, "id")
    0.16296296296296295
    """
    if not all(col in solution.columns for col in ["id", "trigger_words"]):
        raise ValueError("Solution DataFrame must contain 'id' and 'trigger_words' columns.")
    if not all(col in submission.columns for col in ["id", "trigger_words"]):
        raise ValueError("Submission DataFrame must contain 'id' and 'trigger_words' columns.")
    
    def safe_parse_spans(trigger_words):
        if isinstance(trigger_words, str):
            try:
                return ast.literal_eval(trigger_words)
            except (ValueError, SyntaxError):
                return []
        if isinstance(trigger_words, (list, tuple)):
            return trigger_words
        return []

    def extract_tokens_from_spans(spans):
        tokens = set()
        for start, end in spans:
            tokens.update(range(start, end))
        return tokens
    
    solution = solution.copy()
    submission = submission.copy()

    solution["trigger_words"] = solution["trigger_words"].apply(safe_parse_spans)
    submission["trigger_words"] = submission["trigger_words"].apply(safe_parse_spans)

    merged = pd.merge(
        solution,
        submission,
        on="id",
        suffixes=("_solution", "_submission")
    )

    total_true_tokens = 0
    total_pred_tokens = 0
    overlapping_tokens = 0

    for _, row in merged.iterrows():
        true_spans = row["trigger_words_solution"]
        pred_spans = row["trigger_words_submission"]

        true_tokens = extract_tokens_from_spans(true_spans)
        pred_tokens = extract_tokens_from_spans(pred_spans)

        total_true_tokens += len(true_tokens)
        total_pred_tokens += len(pred_tokens)
        overlapping_tokens += len(true_tokens & pred_tokens)

    precision = overlapping_tokens / total_pred_tokens if total_pred_tokens > 0 else 0
    recall = overlapping_tokens / total_true_tokens if total_true_tokens > 0 else 0
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return f1